# Holly Bronze Database - SQL Query Interface

This notebook provides an interactive interface for querying and manipulating data in the `holly_bronze.db` SQLite database.

## Database Contents
The database contains municipal data including:
- Budget data (as_of_6-30-XX tables)
- Road improvement plans (critical_road_improvements_X)
- Water usage data (water_pumped_XXXX-XXXX)
- Capital improvement projects (rowe_cip-X tables)
- Utility budgets (water_25-26, sewer_25-26, etc.)
- And much more!

## 1. Import Required Libraries

In [4]:
import sqlite3
import pandas as pd
from pathlib import Path
import numpy as np
from datetime import datetime

# For better display of DataFrames
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', None)

print("✓ All libraries imported successfully!")

✓ All libraries imported successfully!


## 2. Connect to SQLite Database

In [5]:
# Database configuration
DB_PATH = "/Users/isaackaczor/VS-Code/R-Projects/holly_bronze.db"

def get_connection():
    """Get database connection."""
    if not Path(DB_PATH).exists():
        raise FileNotFoundError(f"Database {DB_PATH} not found!")
    return sqlite3.connect(DB_PATH)

def query(sql, params=None):
    """Execute SQL query and return pandas DataFrame."""
    with get_connection() as conn:
        return pd.read_sql(sql, conn, params=params or ())

def execute(sql, params=None):
    """Execute SQL statement (INSERT/UPDATE/DELETE)."""
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute(sql, params or ())
        conn.commit()
        return cursor.rowcount

# Test connection
try:
    test_query = "SELECT name FROM sqlite_master WHERE type='table' LIMIT 1"
    result = query(test_query)
    print(f"✓ Successfully connected to {DB_PATH}")
    print(f"✓ Database contains {len(query('SELECT name FROM sqlite_master WHERE type=\"table\"'))} tables")
except Exception as e:
    print(f"✗ Connection failed: {e}")

✓ Successfully connected to /Users/isaackaczor/VS-Code/R-Projects/holly_bronze.db
✓ Database contains 29 tables


## 3. Explore Database Schema

In [ ]:
# Get all tables in the database
tables_sql = """
SELECT name as table_name 
FROM sqlite_master 
WHERE type='table' 
ORDER BY name;
"""

tables = query(tables_sql)
print(f"Total tables: {len(tables)}")
print("\nAll tables:")
for i, table in enumerate(tables['table_name'], 1):
    print(f"{i:2d}. {table}")

tables

In [ ]:
# Function to examine table structure
def describe_table(table_name):
    """Show detailed information about a table."""
    print(f"\n=== Table: {table_name} ===")
    
    # Get column information
    columns_sql = f"PRAGMA table_info(`{table_name}`);"
    columns = query(columns_sql)
    
    # Get row count
    count_sql = f"SELECT COUNT(*) as row_count FROM `{table_name}`;"
    count = query(count_sql)['row_count'][0]
    
    print(f"Rows: {count:,}")
    print(f"Columns: {len(columns)}")
    print("\nColumn Details:")
    return columns[['name', 'type', 'notnull', 'pk']]

# Example: Examine critical_road_improvements_1 table
describe_table('critical_road_improvements_1')

## 4. Basic Data Exploration Queries

In [ ]:
# 1. Road Improvements Overview
print("=== ROAD IMPROVEMENTS DATA ===")
road_query = """
SELECT 
    segment_name,
    proposed_treatment,
    estimated_cost,
    current_rating,
    surface_subtype
FROM critical_road_improvements_1 
ORDER BY estimated_cost DESC 
LIMIT 10;
"""
road_data = query(road_query)
print("Top 10 most expensive road improvements:")
road_data

In [ ]:
# 2. Recent Water Usage Data
print("=== WATER USAGE DATA (2024-2025) ===")
water_query = """
SELECT 
    date,
    avg as avg_daily_usage,
    max as max_daily_usage,
    total as monthly_total,
    million_gallons as unit
FROM `water_pumped_2024-2025`
ORDER BY date;
"""
water_data = query(water_query)
print("Monthly water pumping data:")
water_data

In [ ]:
# 3. Budget Overview (General Fund 2025-26)
print("=== BUDGET DATA (2025-26) ===")
budget_query = """
SELECT 
    department_name,
    gl_number,
    account_description,
    `202526_recommended_budget` as recommended_budget
FROM `general_25-26`
WHERE `202526_recommended_budget` > 100000
ORDER BY `202526_recommended_budget` DESC
LIMIT 15;
"""
budget_data = query(budget_query)
print("Largest budget items (>$100k):")
budget_data

## 5. Data Filtering and Selection

Use WHERE clauses to filter data based on specific criteria.

In [ ]:
# Filter roads needing reconstruction (rating ≤ 2)
critical_roads = """
SELECT 
    segment_name,
    current_rating,
    proposed_treatment,
    estimated_cost
FROM critical_road_improvements_1 
WHERE current_rating <= 2 
    AND proposed_treatment LIKE '%Reconstruct%'
ORDER BY estimated_cost DESC;
"""
print("Roads needing reconstruction (rating ≤ 2):")
query(critical_roads)

In [ ]:
# Filter budget items by department and amount
dept_budget = """
SELECT 
    department_name,
    account_description,
    `202425_amended_budget` as current_budget,
    `202526_recommended_budget` as proposed_budget,
    (`202526_recommended_budget` - `202425_amended_budget`) as budget_change
FROM `general_25-26`
WHERE department_name LIKE '%PUBLIC%'
    AND `202526_recommended_budget` > 50000
ORDER BY budget_change DESC;
"""
print("Public department budget changes (>$50k):")
query(dept_budget)

## 6. Data Aggregation and Grouping

Use GROUP BY with aggregate functions to analyze patterns and summaries.

In [ ]:
# Analyze road conditions by rating
road_summary = """
SELECT 
    current_rating,
    COUNT(*) as segment_count,
    ROUND(AVG(estimated_cost), 0) as avg_cost,
    ROUND(SUM(estimated_cost), 0) as total_cost,
    MIN(estimated_cost) as min_cost,
    MAX(estimated_cost) as max_cost
FROM critical_road_improvements_1 
WHERE estimated_cost IS NOT NULL
GROUP BY current_rating
ORDER BY current_rating;
"""
print("Road improvement costs by condition rating:")
query(road_summary)

In [ ]:
# Analyze water usage trends by month
water_summary = """
SELECT 
    strftime('%m', date) as month,
    strftime('%Y', date) as year,
    COUNT(*) as records,
    ROUND(AVG(avg), 2) as avg_daily_usage,
    ROUND(AVG(total), 2) as avg_monthly_total
FROM `water_pumped_2024-2025`
GROUP BY year, month
ORDER BY year, month;
"""
print("Water usage patterns by month:")
query(water_summary)

## 7. Joining Tables

Combine data from multiple tables using JOIN operations.

In [ ]:
# Example: Compare critical road improvements between tables 1 and 2
# First, let's see what's in table 2
print("Sample from critical_road_improvements_2:")
sample2 = query("SELECT * FROM critical_road_improvements_2 LIMIT 5")
print(sample2)

# Note: Actual JOIN depends on common columns between tables
# This is a template - adjust based on actual table structures

## 8. Data Manipulation (UPDATE/INSERT/DELETE)

⚠️ **CAUTION**: These operations modify your database permanently. Consider backing up your data first!

In [ ]:
# Example 1: Create a backup/analysis table
create_analysis_table = """
CREATE TABLE IF NOT EXISTS road_analysis AS
SELECT 
    segment_name,
    current_rating,
    estimated_cost,
    CASE 
        WHEN current_rating <= 2 THEN 'Critical'
        WHEN current_rating <= 4 THEN 'Moderate'
        ELSE 'Good'
    END as condition_category
FROM critical_road_improvements_1;
"""

# Uncomment to execute:
# execute(create_analysis_table)
# print("Analysis table created!")

print("Example CREATE TABLE statement (commented out for safety)")
print(create_analysis_table)

## 9. Advanced Query Examples

Complex queries using subqueries, window functions, and CTEs.

In [ ]:
# Advanced Query 1: Ranking roads by cost within each condition rating
ranking_query = """
SELECT 
    segment_name,
    current_rating,
    estimated_cost,
    RANK() OVER (PARTITION BY current_rating ORDER BY estimated_cost DESC) as cost_rank_in_rating,
    ROUND(100.0 * estimated_cost / SUM(estimated_cost) OVER (PARTITION BY current_rating), 1) as pct_of_rating_cost
FROM critical_road_improvements_1
WHERE estimated_cost IS NOT NULL
ORDER BY current_rating, cost_rank_in_rating;
"""
print("Roads ranked by cost within each condition rating:")
query(ranking_query)

In [ ]:
# Advanced Query 2: Using CTE for budget year-over-year comparison
cte_budget_query = """
WITH budget_changes AS (
    SELECT 
        department_name,
        account_description,
        `202425_amended_budget` as current_budget,
        `202526_recommended_budget` as proposed_budget,
        (`202526_recommended_budget` - `202425_amended_budget`) as budget_change,
        CASE 
            WHEN `202425_amended_budget` > 0 THEN 
                ROUND(100.0 * (`202526_recommended_budget` - `202425_amended_budget`) / `202425_amended_budget`, 1)
            ELSE NULL 
        END as pct_change
    FROM `general_25-26`
    WHERE `202425_amended_budget` IS NOT NULL 
        AND `202526_recommended_budget` IS NOT NULL
)
SELECT 
    department_name,
    COUNT(*) as line_items,
    ROUND(SUM(current_budget), 0) as total_current,
    ROUND(SUM(proposed_budget), 0) as total_proposed,
    ROUND(SUM(budget_change), 0) as total_change,
    ROUND(AVG(pct_change), 1) as avg_pct_change
FROM budget_changes 
WHERE ABS(budget_change) > 1000
GROUP BY department_name
HAVING COUNT(*) > 2
ORDER BY total_change DESC;
"""
print("Department budget changes (CTE example):")
query(cte_budget_query)

## 10. Export Query Results

Save your analysis results to files for sharing or further processing.

In [ ]:
# Export functions
def export_query_to_csv(sql, filename):
    """Export query results to CSV file."""
    df = query(sql)
    df.to_csv(filename, index=False)
    print(f"Exported {len(df)} rows to {filename}")
    return df

def export_query_to_excel(sql, filename, sheet_name='Sheet1'):
    """Export query results to Excel file."""
    df = query(sql)
    df.to_excel(filename, sheet_name=sheet_name, index=False)
    print(f"Exported {len(df)} rows to {filename}")
    return df

# Example exports (uncomment to use):
# road_summary_sql = "SELECT * FROM critical_road_improvements_1 ORDER BY estimated_cost DESC"
# export_query_to_csv(road_summary_sql, "road_improvements_summary.csv")
# export_query_to_excel(road_summary_sql, "road_improvements_summary.xlsx", "Road_Data")

print("Export functions defined!")
print("Use export_query_to_csv() or export_query_to_excel() with your SQL queries.")

## 🚀 Your Custom Query Workspace

Use the cells below to write your own SQL queries and analysis!

In [ ]:
# Write your custom SQL queries here
# Example:
# my_query = """
# SELECT column1, column2 
# FROM table_name 
# WHERE condition
# """
# result = query(my_query)
# print(result)

# Quick helper: See all available tables
# query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name")

## 🔥 Quick SQL Execution

For writing SQL queries without the Python wrapper, use these simplified approaches:

In [ ]:
# Method 1: Single-line SQL execution
def sql(query_string):
    """Execute SQL directly - just type: sql("SELECT * FROM table_name")"""
    return query(query_string)

# Method 2: Multi-line SQL with triple quotes
def run_sql():
    """Paste your SQL between the triple quotes below and run this cell"""
    sql_command = """
    
    SELECT * FROM critical_road_improvements_1 
    WHERE current_rating <= 3 
    LIMIT 5;
    
    """
    return query(sql_command.strip())

# Example usage:
print("Quick SQL methods available:")
print("1. sql('SELECT * FROM table_name LIMIT 5')")
print("2. Modify the run_sql() function above and run the cell")
print("3. Use the SQLite extension (right-click on holly_bronze.db in Explorer)")
print("4. Use terminal: sqlite3 holly_bronze.db")

# Demo:
sql("SELECT COUNT(*) as total_tables FROM sqlite_master WHERE type='table'")

In [ ]:
# ✏️ DIRECT SQL WORKSPACE - Edit the SQL below and run this cell

my_sql = """
SELECT 
    segment_name,
    current_rating,
    estimated_cost
FROM critical_road_improvements_1 
WHERE estimated_cost > 200000
ORDER BY estimated_cost DESC
LIMIT 10;
"""

# Execute the SQL
query(my_sql)

## 🛠️ Creating and Editing Tables

Yes! You can create, modify, and manage tables using any of the methods. Here are safe examples:

In [ ]:
# 1. CREATE TABLE Examples

# Example 1: Create a simple analysis table
create_table_sql = """
CREATE TABLE IF NOT EXISTS road_priorities (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    segment_name TEXT NOT NULL,
    priority_score INTEGER,
    notes TEXT,
    created_date DATE DEFAULT CURRENT_DATE
);
"""

# Example 2: Create table from existing data (CTAS - Create Table As Select)
create_summary_table = """
CREATE TABLE IF NOT EXISTS budget_summary_2025 AS
SELECT 
    department_name,
    COUNT(*) as line_items,
    ROUND(SUM(`202526_recommended_budget`), 0) as total_budget,
    ROUND(AVG(`202526_recommended_budget`), 0) as avg_budget
FROM `general_25-26`
WHERE `202526_recommended_budget` > 0
GROUP BY department_name;
"""

print("Table creation examples:")
print("1. CREATE TABLE road_priorities - Manual structure")
print("2. CREATE TABLE budget_summary_2025 - From existing data")
print("\nTo execute, uncomment the lines below:")
print("# execute(create_table_sql)")
print("# execute(create_summary_table)")

# Uncomment to create tables:
# execute(create_table_sql)
# execute(create_summary_table)
# print("✓ Tables created!")

In [ ]:
# 2. INSERT DATA Examples

# Example 1: Insert single row
insert_single = """
INSERT INTO road_priorities (segment_name, priority_score, notes)
VALUES ('Main Street', 10, 'High traffic area needing immediate attention');
"""

# Example 2: Insert multiple rows
insert_multiple = """
INSERT INTO road_priorities (segment_name, priority_score, notes) VALUES
('Oak Avenue', 8, 'Moderate repairs needed'),
('Pine Street', 6, 'Scheduled for next year'),
('Elm Drive', 9, 'Safety concerns reported');
"""

# Example 3: Insert from SELECT (populate from existing data)
insert_from_select = """
INSERT INTO road_priorities (segment_name, priority_score, notes)
SELECT 
    segment_name,
    CASE 
        WHEN current_rating <= 2 THEN 10
        WHEN current_rating <= 3 THEN 7
        ELSE 4
    END as priority_score,
    'Auto-generated from road improvements data' as notes
FROM critical_road_improvements_1
WHERE estimated_cost > 100000
LIMIT 5;
"""

print("INSERT examples:")
print("1. Single row insert")
print("2. Multiple rows insert")  
print("3. Insert from existing data")
print("\nUncomment to execute:")
print("# execute(insert_single)")
print("# execute(insert_multiple)")
print("# execute(insert_from_select)")

In [ ]:
# 3. UPDATE and DELETE Examples

# Example 1: Update specific records
update_example = """
UPDATE road_priorities 
SET priority_score = 9, 
    notes = 'Updated priority - resident complaints received'
WHERE segment_name = 'Oak Avenue';
"""

# Example 2: Bulk update based on conditions
bulk_update = """
UPDATE road_priorities 
SET priority_score = priority_score + 1
WHERE priority_score < 5 
    AND notes LIKE '%next year%';
"""

# Example 3: Safe delete with WHERE clause (NEVER delete without WHERE!)
delete_example = """
DELETE FROM road_priorities 
WHERE priority_score < 3 
    AND created_date < date('now', '-30 days');
"""

# Example 4: Update using JOIN with existing data
update_with_join = """
UPDATE road_priorities 
SET notes = 'Estimated cost: $' || 
    (SELECT estimated_cost 
     FROM critical_road_improvements_1 cr
     WHERE cr.segment_name = road_priorities.segment_name
     LIMIT 1)
WHERE EXISTS (
    SELECT 1 FROM critical_road_improvements_1 cr2
    WHERE cr2.segment_name = road_priorities.segment_name
);
"""

print("UPDATE/DELETE examples:")
print("1. Update specific records")
print("2. Bulk updates")
print("3. Safe deletes (always use WHERE!)")
print("4. Update using data from other tables")
print("\n⚠️ WARNING: These modify data permanently!")
print("Uncomment carefully:")
print("# execute(update_example)")
print("# execute(bulk_update)")
print("# execute(delete_example)")
print("# execute(update_with_join)")

In [ ]:
# 4. ALTER TABLE Examples (Modify Table Structure)

# Example 1: Add new columns
add_column = """
ALTER TABLE road_priorities 
ADD COLUMN estimated_completion_date DATE;
"""

add_multiple_columns = """
-- Note: SQLite requires separate ALTER statements for each column
-- ALTER TABLE road_priorities ADD COLUMN budget_allocated INTEGER;
-- ALTER TABLE road_priorities ADD COLUMN contractor_assigned TEXT;
"""

# Example 2: Create indexes for better performance
create_index = """
CREATE INDEX IF NOT EXISTS idx_road_priorities_score 
ON road_priorities(priority_score);
"""

# Example 3: Create a view (virtual table)
create_view = """
CREATE VIEW IF NOT EXISTS high_priority_roads AS
SELECT 
    rp.segment_name,
    rp.priority_score,
    rp.notes,
    cr.estimated_cost,
    cr.current_rating
FROM road_priorities rp
LEFT JOIN critical_road_improvements_1 cr ON rp.segment_name = cr.segment_name
WHERE rp.priority_score >= 8;
"""

print("ALTER TABLE examples:")
print("1. Add new columns")
print("2. Create indexes for performance")
print("3. Create views (virtual tables)")
print("\nNote: SQLite has limited ALTER TABLE support")
print("For major changes, create new table and copy data")
print("\nUncomment to execute:")
print("# execute(add_column)")
print("# execute(create_index)")
print("# execute(create_view)")

In [ ]:
# 5. 🎯 DIRECT TABLE OPERATION WORKSPACE

# Edit the SQL below for your table operations:
table_operation_sql = """

-- Example: Create a simple tracking table
CREATE TABLE IF NOT EXISTS project_tracker (
    id INTEGER PRIMARY KEY,
    project_name TEXT,
    status TEXT CHECK(status IN ('Planning', 'In Progress', 'Completed')),
    start_date DATE,
    budget DECIMAL(10,2)
);

"""

print("Ready to execute table operations!")
print("Edit the SQL above, then uncomment the line below:")
print("# execute(table_operation_sql.strip())")

# Uncomment to run:
# execute(table_operation_sql.strip())

## 💾 Temporary Tables

Temporary tables are perfect for analysis - they exist only during your session and are automatically deleted when you disconnect!

In [ ]:
# 1. CREATE TEMPORARY TABLE Examples

# Example 1: Simple temporary analysis table
create_temp_analysis = """
CREATE TEMPORARY TABLE temp_road_analysis AS
SELECT 
    segment_name,
    current_rating,
    estimated_cost,
    CASE 
        WHEN current_rating <= 2 THEN 'Critical'
        WHEN current_rating <= 4 THEN 'Moderate'
        ELSE 'Good'
    END as priority_level,
    CASE 
        WHEN estimated_cost > 300000 THEN 'High Cost'
        WHEN estimated_cost > 100000 THEN 'Medium Cost'
        ELSE 'Low Cost'
    END as cost_category
FROM critical_road_improvements_1;
"""

# Example 2: Temporary table for budget analysis
create_temp_budget = """
CREATE TEMPORARY TABLE temp_budget_changes AS
SELECT 
    department_name,
    COUNT(*) as line_items,
    ROUND(SUM(`202425_amended_budget`), 0) as total_current,
    ROUND(SUM(`202526_recommended_budget`), 0) as total_proposed,
    ROUND(SUM(`202526_recommended_budget` - `202425_amended_budget`), 0) as net_change,
    ROUND(100.0 * SUM(`202526_recommended_budget` - `202425_amended_budget`) / 
          NULLIF(SUM(`202425_amended_budget`), 0), 1) as pct_change
FROM `general_25-26`
WHERE `202425_amended_budget` IS NOT NULL 
    AND `202526_recommended_budget` IS NOT NULL
GROUP BY department_name
HAVING ABS(net_change) > 5000;
"""

print("TEMPORARY TABLE examples:")
print("✓ Automatically cleaned up when session ends")
print("✓ Perfect for analysis without cluttering your database")
print("✓ Can be joined with regular tables")
print("\nUncomment to create:")
print("# execute(create_temp_analysis)")
print("# execute(create_temp_budget)")

# Uncomment to create temporary tables:
# execute(create_temp_analysis)
# execute(create_temp_budget)
# print("✓ Temporary tables created!")

In [ ]:
# 2. Working with Temporary Tables

# View temporary table contents
view_temp_roads = """
SELECT 
    priority_level,
    cost_category,
    COUNT(*) as segment_count,
    ROUND(AVG(estimated_cost), 0) as avg_cost,
    ROUND(SUM(estimated_cost), 0) as total_cost
FROM temp_road_analysis
GROUP BY priority_level, cost_category
ORDER BY priority_level, cost_category;
"""

# Join temporary table with regular tables
join_temp_analysis = """
SELECT 
    tra.segment_name,
    tra.priority_level,
    tra.cost_category,
    tra.estimated_cost,
    cr2.proposed_treatment as treatment_plan
FROM temp_road_analysis tra
LEFT JOIN critical_road_improvements_2 cr2 
    ON tra.segment_name = cr2.segment_name
WHERE tra.priority_level = 'Critical'
ORDER BY tra.estimated_cost DESC;
"""

# Complex analysis using temporary table
complex_temp_analysis = """
SELECT 
    tb.department_name,
    tb.net_change,
    tb.pct_change,
    CASE 
        WHEN tb.pct_change > 10 THEN 'Significant Increase'
        WHEN tb.pct_change > 0 THEN 'Moderate Increase'
        WHEN tb.pct_change > -10 THEN 'Moderate Decrease'
        ELSE 'Significant Decrease'
    END as change_category
FROM temp_budget_changes tb
WHERE ABS(tb.net_change) > 10000
ORDER BY tb.net_change DESC;
"""

print("Working with temporary tables:")
print("1. Aggregate and analyze temp data")
print("2. Join temps with regular tables")  
print("3. Complex multi-step analysis")
print("\nRun queries on temporary tables:")
print("# query(view_temp_roads)")
print("# query(join_temp_analysis)")
print("# query(complex_temp_analysis)")

# Example: Check if temp tables exist
check_temps = """
SELECT name, type 
FROM sqlite_temp_master 
WHERE type = 'table'
ORDER BY name;
"""
print(f"\nCurrent temporary tables:")
try:
    temp_tables = query(check_temps)
    if len(temp_tables) > 0:
        print(temp_tables)
    else:
        print("No temporary tables found (create some first!)")
except:
    print("No temporary tables found (create some first!)")

# Creating Detail Sheet

Joining the various 'As of...' sheeets and the new budgets to create a comprehensive spending history and including the budget for the coming financial year

In [50]:
# First, let's examine the current issue with department_number and gl_number
diagnostic_sql = """
SELECT 
    department_number,
    gl_number,
    typeof(department_number) as dept_type,
    typeof(gl_number) as gl_type,
    length(CAST(department_number AS TEXT)) as dept_length,
    length(CAST(gl_number AS TEXT)) as gl_length
FROM as_of_6_30_24
LIMIT 10;
"""

print("Current state of department_number and gl_number columns:")
query(diagnostic_sql)

Current state of department_number and gl_number columns:


,department_number,gl_number,dept_type,gl_type,dept_length,gl_length
0,0,402.0,integer,real,1,5
1,0,411.0,integer,real,1,5
2,0,434.0,integer,real,1,5
3,0,439.0,integer,real,1,5
4,0,445.0,integer,real,1,5
5,0,447.0,integer,real,1,5
6,0,476.0,integer,real,1,5
7,0,477.0,integer,real,1,5
8,0,478.0,integer,real,1,5
9,0,479.0,integer,real,1,5


## 🔢➡️📝 Fixing Numeric Columns: Adding Leading Zeros

**Problem**: Department numbers and GL numbers are stored as integers, but they should be text with leading zeros (e.g., "001" instead of 1).

**Solutions**: We'll pad these columns with leading zeros to restore their proper format.

In [51]:
# Step 1: Identify columns that need leading zero padding

def find_numeric_code_columns():
    """Find columns that look like they should have leading zeros."""
    
    # Common patterns for columns that need leading zeros
    code_column_patterns = [
        'department_number', 'dept_number', 'gl_number', 'account_number', 
        'fund_number', 'object_code', 'cost_center', 'project_code'
    ]
    
    all_tables = query("SELECT name FROM sqlite_master WHERE type='table'")
    columns_to_fix = []
    
    for table_name in all_tables['name']:
        try:
            # Get column info for this table
            columns_info = query(f"PRAGMA table_info(`{table_name}`)")
            
            for _, col in columns_info.iterrows():
                col_name = col['name'].lower()
                col_type = col['type'].upper()
                
                # Check if this looks like a code column that might need padding
                for pattern in code_column_patterns:
                    if pattern in col_name and ('INT' in col_type or 'NUM' in col_type or col_type == ''):
                        # Sample some data to see if it needs padding
                        sample_query = f"""
                        SELECT 
                            `{col['name']}`,
                            length(CAST(`{col['name']}` AS TEXT)) as current_length,
                            COUNT(*) as count
                        FROM `{table_name}` 
                        WHERE `{col['name']}` IS NOT NULL
                        GROUP BY `{col['name']}`, length(CAST(`{col['name']}` AS TEXT))
                        ORDER BY current_length, `{col['name']}`
                        LIMIT 10
                        """
                        
                        try:
                            sample_data = query(sample_query)
                            if len(sample_data) > 0:
                                min_length = sample_data['current_length'].min()
                                max_length = sample_data['current_length'].max()
                                
                                # If we have varying lengths and some are short, suggest padding
                                if min_length < max_length and min_length <= 2:
                                    columns_to_fix.append({
                                        'table': table_name,
                                        'column': col['name'],
                                        'current_type': col['type'],
                                        'min_length': min_length,
                                        'max_length': max_length,
                                        'suggested_padding': max_length
                                    })
                        except:
                            pass
                        break
        except Exception as e:
            print(f"Error checking table {table_name}: {e}")
    
    return columns_to_fix

# Find columns that need fixing
columns_needing_padding = find_numeric_code_columns()

print(f"Found {len(columns_needing_padding)} columns that may need leading zero padding:")
for col in columns_needing_padding:
    print(f"  Table: {col['table']} | Column: {col['column']} | Lengths: {col['min_length']}-{col['max_length']} | Suggested padding: {col['suggested_padding']}")

# Let's also check specific known problematic columns
known_problem_columns = ['department_number', 'gl_number', 'fund_number']
print(f"\nChecking specific known problem columns: {known_problem_columns}")

for table_name in ['as_of_6_30_24', 'as_of_6_30_25', 'general_25_26']:  # Common table names after renaming
    try:
        # Check if table exists (might have hyphens still)
        table_check = query(f"SELECT name FROM sqlite_master WHERE name = '{table_name}' OR name = '{table_name.replace('_', '-')}'")
        if len(table_check) == 0:
            continue
            
        actual_table_name = table_check['name'][0]
        
        for col_name in known_problem_columns:
            try:
                check_query = f"""
                SELECT 
                    '{actual_table_name}' as table_name,
                    '{col_name}' as column_name,
                    MIN(`{col_name}`) as min_value,
                    MAX(`{col_name}`) as max_value,
                    MIN(length(CAST(`{col_name}` AS TEXT))) as min_length,
                    MAX(length(CAST(`{col_name}` AS TEXT))) as max_length,
                    COUNT(DISTINCT `{col_name}`) as unique_values
                FROM `{actual_table_name}`
                WHERE `{col_name}` IS NOT NULL
                """
                
                result = query(check_query)
                if len(result) > 0:
                    print(f"\n{actual_table_name}.{col_name}:")
                    print(result)
            except:
                pass
    except:
        pass

Found 2 columns that may need leading zero padding:
  Table: as_of_6_30_24 | Column: department_number | Lengths: 1-3 | Suggested padding: 3
  Table: as_of_6_30_25 | Column: department_number | Lengths: 1-3 | Suggested padding: 3

Checking specific known problem columns: ['department_number', 'gl_number', 'fund_number']

as_of_6_30_24.department_number:
      table_name        column_name  min_value  max_value  min_length  \
0  as_of_6_30_24  department_number          0        905           1   

   max_length  unique_values  
0           3             32  

as_of_6_30_24.gl_number:
      table_name column_name  min_value  max_value  min_length  max_length  \
0  as_of_6_30_24   gl_number      402.0      993.0           5           7   

   unique_values  
0            155  

as_of_6_30_24.fund_number:
      table_name  column_name  min_value  max_value  min_length  max_length  \
0  as_of_6_30_24  fund_number        101        905           3           3   

   unique_values  
0       

In [52]:
# Step 2: Solutions for adding leading zeros

def pad_column_with_zeros(table_name, column_name, target_length=3):
    """
    Update a column to add leading zeros using SQL PRINTF function.
    
    Args:
        table_name: Name of the table
        column_name: Name of the column to pad
        target_length: Target length for the padded string (default: 3)
    """
    
    # Create the UPDATE statement using PRINTF to pad with zeros
    update_sql = f"""
    UPDATE `{table_name}` 
    SET `{column_name}` = PRINTF('%0{target_length}d', CAST(`{column_name}` AS INTEGER))
    WHERE `{column_name}` IS NOT NULL;
    """
    
    return update_sql

def create_padded_view(table_name, columns_to_pad, view_name_suffix="_padded"):
    """
    Create a view with properly padded columns instead of modifying the original table.
    This is a safer approach that doesn't modify your original data.
    """
    
    # Get all columns from the table
    columns_info = query(f"PRAGMA table_info(`{table_name}`)")
    
    select_columns = []
    for _, col in columns_info.iterrows():
        col_name = col['name']
        if col_name in columns_to_pad:
            # Pad this column
            target_length = columns_to_pad[col_name]
            padded_expr = f"PRINTF('%0{target_length}d', CAST(`{col_name}` AS INTEGER)) AS `{col_name}`"
            select_columns.append(padded_expr)
        else:
            # Keep column as-is
            select_columns.append(f"`{col_name}`")
    
    view_name = f"{table_name}{view_name_suffix}"
    create_view_sql = f"""
    CREATE OR REPLACE VIEW `{view_name}` AS
    SELECT 
        {',\n        '.join(select_columns)}
    FROM `{table_name}`;
    """
    
    return create_view_sql, view_name

# Example usage and testing
print("🛠️ PADDING SOLUTIONS:")
print("="*50)

# Solution 1: Direct UPDATE (modifies original data - be careful!)
print("\n1️⃣ DIRECT UPDATE (Permanent changes):")
example_update = pad_column_with_zeros('as_of_6_30_24', 'department_number', 3)
print("Example UPDATE command:")
print(example_update)

# Solution 2: Create padded views (safer - doesn't modify original)
print("\n2️⃣ CREATE PADDED VIEW (Safe - preserves original):")
columns_to_pad = {
    'department_number': 3,  # Pad to 3 digits
    'gl_number': 4,          # Pad to 4 digits  
    'fund_number': 3         # Pad to 3 digits
}

try:
    # Check if as_of_6_30_24 exists (with or without hyphens)
    table_check = query("SELECT name FROM sqlite_master WHERE name LIKE '%as_of_6%30%24%'")
    if len(table_check) > 0:
        actual_table = table_check['name'][0]
        view_sql, view_name = create_padded_view(actual_table, columns_to_pad)
        print(f"Example VIEW creation for {actual_table}:")
        print(view_sql[:500] + "..." if len(view_sql) > 500 else view_sql)
    else:
        print("Table as_of_6_30_24 not found (may need to check exact name)")
except Exception as e:
    print(f"Error creating example view: {e}")

print("\n⚠️ CHOOSE YOUR APPROACH:")
print("• Direct UPDATE: Fast, but permanently changes your data") 
print("• CREATE VIEW: Safe, preserves original data, creates new padded view")
print("• Recommendation: Try views first, then update if you're confident")

🛠️ PADDING SOLUTIONS:

1️⃣ DIRECT UPDATE (Permanent changes):
Example UPDATE command:

    UPDATE `as_of_6_30_24` 
    SET `department_number` = PRINTF('%03d', CAST(`department_number` AS INTEGER))
    WHERE `department_number` IS NOT NULL;
    

2️⃣ CREATE PADDED VIEW (Safe - preserves original):
Example VIEW creation for as_of_6_30_24:

    CREATE OR REPLACE VIEW `as_of_6_30_24_padded` AS
    SELECT 
        `fund_name`,
        PRINTF('%03d', CAST(`fund_number` AS INTEGER)) AS `fund_number`,
        `department_name`,
        PRINTF('%03d', CAST(`department_number` AS INTEGER)) AS `department_number`,
        PRINTF('%04d', CAST(`gl_number` AS INTEGER)) AS `gl_number`,
        `description`,
        `202324__amended_budget`,
        `yeartodate_thru_063024`,
        `available__balance_`,
        `202425__orig_budget`
    FRO...

⚠️ CHOOSE YOUR APPROACH:
• Direct UPDATE: Fast, but permanently changes your data
• CREATE VIEW: Safe, preserves original data, creates new padded view
• R

In [53]:
# Step 3: Comprehensive Fix - All Tables

def fix_all_padding_issues():
    """
    Comprehensive function to fix padding issues across all relevant tables.
    Creates both views (safe) and provides update commands (permanent).
    """
    
    # Define standard padding rules
    padding_rules = {
        'department_number': 3,
        'gl_number': 4, 
        'fund_number': 3,
        'object_code': 4,
        'cost_center': 3,
        'account_number': 6
    }
    
    # Get all tables that might need fixing
    all_tables = query("SELECT name FROM sqlite_master WHERE type='table'")
    
    views_created = []
    update_commands = []
    
    for table_name in all_tables['name']:
        try:
            # Get columns for this table
            columns_info = query(f"PRAGMA table_info(`{table_name}`)")
            
            # Find which columns in this table need padding
            table_columns_to_pad = {}
            for _, col in columns_info.iterrows():
                col_name = col['name']
                if col_name in padding_rules:
                    table_columns_to_pad[col_name] = padding_rules[col_name]
            
            if table_columns_to_pad:
                print(f"\n📋 Table: {table_name}")
                print(f"   Columns to pad: {table_columns_to_pad}")
                
                # Create padded view (safe approach)
                try:
                    view_sql, view_name = create_padded_view(table_name, table_columns_to_pad)
                    views_created.append({
                        'table': table_name,
                        'view': view_name, 
                        'columns': table_columns_to_pad,
                        'sql': view_sql
                    })
                    print(f"   ✓ View ready: {view_name}")
                except Exception as e:
                    print(f"   ✗ View creation failed: {e}")
                
                # Generate update commands (permanent changes)
                for col_name, pad_length in table_columns_to_pad.items():
                    update_cmd = pad_column_with_zeros(table_name, col_name, pad_length)
                    update_commands.append({
                        'table': table_name,
                        'column': col_name,
                        'command': update_cmd
                    })
        
        except Exception as e:
            print(f"Error processing table {table_name}: {e}")
    
    return views_created, update_commands

# Execute the analysis
print("🔍 ANALYZING ALL TABLES FOR PADDING ISSUES...")
print("="*50)

views_to_create, updates_to_run = fix_all_padding_issues()

print(f"\n📊 SUMMARY:")
print(f"   • Found {len(views_to_create)} tables that need padding fixes")
print(f"   • Generated {len(updates_to_run)} update commands")

# Show what we found
if views_to_create:
    print(f"\n🎯 TABLES REQUIRING FIXES:")
    for view_info in views_to_create:
        print(f"   • {view_info['table']} → {view_info['view']} (columns: {list(view_info['columns'].keys())})")

print(f"\n📋 NEXT STEPS:")
print(f"   1. Create safe padded views (recommended first)")
print(f"   2. Test queries with the new views")  
print(f"   3. If satisfied, run permanent updates")
print(f"   4. Drop views and use updated tables")

🔍 ANALYZING ALL TABLES FOR PADDING ISSUES...

📋 Table: general_25_26
   Columns to pad: {'gl_number': 4}
   ✓ View ready: general_25_26_padded

📋 Table: major_street_25_26
   Columns to pad: {'gl_number': 4}
   ✓ View ready: major_street_25_26_padded

📋 Table: local_street_25_26
   Columns to pad: {'gl_number': 4}
   ✓ View ready: local_street_25_26_padded

📋 Table: lake_improvement_25_26
   Columns to pad: {'gl_number': 4}
   ✓ View ready: lake_improvement_25_26_padded

📋 Table: refuse_25_26
   Columns to pad: {'gl_number': 4}
   ✓ View ready: refuse_25_26_padded

📋 Table: sewer_25_26
   Columns to pad: {'gl_number': 4}
   ✓ View ready: sewer_25_26_padded

📋 Table: water_25_26
   Columns to pad: {'gl_number': 4}
   ✓ View ready: water_25_26_padded

📋 Table: as_of_6_30_21
   Columns to pad: {'gl_number': 4}
   ✓ View ready: as_of_6_30_21_padded

📋 Table: as_of_6_30_22
   Columns to pad: {'gl_number': 4}
   ✓ View ready: as_of_6_30_22_padded

📋 Table: as_of_6_30_23
   Columns to pad: {'

In [54]:
# Step 4: EXECUTION FUNCTIONS

def create_all_padded_views():
    """Create all the padded views (safe approach)."""
    success_count = 0
    
    print("🏗️ CREATING PADDED VIEWS...")
    print("="*40)
    
    for view_info in views_to_create:
        try:
            execute(view_info['sql'])
            print(f"✓ Created view: {view_info['view']}")
            success_count += 1
            
            # Test the view with a sample query
            test_query = f"SELECT * FROM `{view_info['view']}` LIMIT 3"
            sample = query(test_query)
            print(f"  Sample data preview:")
            for col in view_info['columns'].keys():
                if col in sample.columns:
                    values = sample[col].head(3).tolist()
                    print(f"    {col}: {values}")
            print()
            
        except Exception as e:
            print(f"✗ Failed to create {view_info['view']}: {e}")
    
    print(f"✅ Successfully created {success_count}/{len(views_to_create)} views")
    return success_count

def apply_all_updates():
    """Apply all the permanent updates (modifies original data)."""
    success_count = 0
    
    print("⚠️ APPLYING PERMANENT UPDATES...")
    print("="*40)
    print("⚠️ WARNING: This will permanently modify your original data!")
    print("⚠️ Make sure you've tested with views first!")
    print()
    
    for update_info in updates_to_run:
        try:
            result = execute(update_info['command'])
            print(f"✓ Updated {update_info['table']}.{update_info['column']} ({result} rows)")
            success_count += 1
        except Exception as e:
            print(f"✗ Failed to update {update_info['table']}.{update_info['column']}: {e}")
    
    print(f"\n✅ Successfully applied {success_count}/{len(updates_to_run)} updates")
    return success_count

def show_before_after_comparison(table_name, columns=['department_number', 'gl_number']):
    """Show before/after comparison for a specific table."""
    
    print(f"📊 BEFORE/AFTER COMPARISON: {table_name}")
    print("="*50)
    
    # Check if padded view exists
    view_name = f"{table_name}_padded"
    view_check = query(f"SELECT name FROM sqlite_master WHERE name = '{view_name}'")
    
    if len(view_check) == 0:
        print(f"❌ Padded view {view_name} not found. Create views first.")
        return
    
    try:
        # Original table
        original_cols = [f"`{col}`" for col in columns if col in query(f"PRAGMA table_info(`{table_name}`)").name.tolist()]
        if original_cols:
            original_query = f"SELECT {', '.join(original_cols)} FROM `{table_name}` LIMIT 5"
            original_data = query(original_query)
            
            print("🔴 ORIGINAL (with missing leading zeros):")
            print(original_data)
        
        # Padded view  
        padded_cols = [f"`{col}`" for col in columns]
        padded_query = f"SELECT {', '.join(padded_cols)} FROM `{view_name}` LIMIT 5"
        padded_data = query(padded_query)
        
        print(f"\n🟢 PADDED (with leading zeros):")
        print(padded_data)
        
    except Exception as e:
        print(f"Error in comparison: {e}")

# EXECUTION CONTROLS
print("🚀 READY TO FIX LEADING ZERO ISSUES")
print("="*50)
print("Available functions:")
print("• create_all_padded_views() - Create safe views with padded columns")
print("• show_before_after_comparison('table_name') - Compare original vs padded")
print("• apply_all_updates() - Apply permanent changes (use with caution!)")
print()
print("Recommended workflow:")
print("1. create_all_padded_views()")
print("2. show_before_after_comparison('as_of_6_30_24')")  
print("3. Test your queries with the new _padded views")
print("4. If satisfied: apply_all_updates()")
print()
print("UNCOMMENT TO EXECUTE:")
print("# create_all_padded_views()")
print("# show_before_after_comparison('as_of_6_30_24')")
print("# apply_all_updates()  # Only after testing!")

# AUTO-EXECUTION: Run the analysis and create views
print("\n🚀 RUNNING AUTOMATIC SETUP...")
print("="*40)

# Always run the analysis fresh to avoid variable scope issues
print("📋 Running fresh analysis to find tables needing padding...")

try:
    views_to_create, updates_to_run = fix_all_padding_issues()
    
    # Now create the views if we found any
    if views_to_create and len(views_to_create) > 0:
        print(f"\n🏗️ Creating {len(views_to_create)} padded views automatically...")
        
        # Create each view individually with error handling
        success_count = 0
        for view_info in views_to_create:
            try:
                execute(view_info['sql'])
                print(f"✓ Created view: {view_info['view']}")
                success_count += 1
                
                # Test the view with a sample query
                test_query = f"SELECT * FROM `{view_info['view']}` LIMIT 3"
                sample = query(test_query)
                print(f"  Sample data preview:")
                for col in view_info['columns'].keys():
                    if col in sample.columns:
                        values = sample[col].head(3).tolist()
                        print(f"    {col}: {values}")
                print()
                
            except Exception as e:
                print(f"✗ Failed to create {view_info['view']}: {e}")
        
        print(f"✅ Successfully created {success_count}/{len(views_to_create)} views")
        
        # Show comparison for the first successful table
        if success_count > 0:
            print("\n📊 Showing before/after comparison...")
            first_table = views_to_create[0]['table']
            
            # Manual comparison since the function might have issues
            print(f"📊 COMPARISON FOR: {first_table}")
            print("="*50)
            
            try:
                # Original table data
                original_query = f"SELECT department_number, gl_number, fund_number FROM `{first_table}` LIMIT 5"
                original_data = query(original_query)
                print("🔴 ORIGINAL (with missing leading zeros):")
                print(original_data)
                
                # Padded view data
                view_name = f"{first_table}_padded"
                padded_query = f"SELECT department_number, gl_number, fund_number FROM `{view_name}` LIMIT 5"
                padded_data = query(padded_query)
                print(f"\n🟢 PADDED (with leading zeros):")
                print(padded_data)
                
            except Exception as e:
                print(f"Error in comparison: {e}")
        
    else:
        print("ℹ️ No tables found that need padding fixes.")
        print("This might mean:")
        print("  • Your tables already have proper leading zeros")
        print("  • The table names have changed (check table names)")
        print("  • The columns aren't named as expected")
        
        # Let's check what tables actually exist
        print("\n🔍 Let's check what tables exist:")
        all_tables = query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name")
        print("Available tables:")
        for i, table_name in enumerate(all_tables['name'], 1):
            print(f"  {i}. {table_name}")
        
        # Check a specific table to see its structure
        if len(all_tables) > 0:
            sample_table = all_tables['name'].iloc[0]
            print(f"\nSample table structure ({sample_table}):")
            try:
                columns = query(f"PRAGMA table_info(`{sample_table}`)")
                print(columns[['name', 'type']])
            except Exception as e:
                print(f"Error checking table structure: {e}")

except Exception as e:
    print(f"❌ Error in analysis: {e}")
    print("This might be a function definition issue. Please run the earlier cells first.")

🚀 READY TO FIX LEADING ZERO ISSUES
Available functions:
• create_all_padded_views() - Create safe views with padded columns
• show_before_after_comparison('table_name') - Compare original vs padded
• apply_all_updates() - Apply permanent changes (use with caution!)

Recommended workflow:
1. create_all_padded_views()
2. show_before_after_comparison('as_of_6_30_24')
3. Test your queries with the new _padded views
4. If satisfied: apply_all_updates()

UNCOMMENT TO EXECUTE:
# create_all_padded_views()
# show_before_after_comparison('as_of_6_30_24')
# apply_all_updates()  # Only after testing!

🚀 RUNNING AUTOMATIC SETUP...
📋 Running fresh analysis to find tables needing padding...

📋 Table: general_25_26
   Columns to pad: {'gl_number': 4}
   ✓ View ready: general_25_26_padded

📋 Table: major_street_25_26
   Columns to pad: {'gl_number': 4}
   ✓ View ready: major_street_25_26_padded

📋 Table: local_street_25_26
   Columns to pad: {'gl_number': 4}
   ✓ View ready: local_street_25_26_padded



In [55]:
# 🔧 SIMPLE DIAGNOSTIC - Step by Step Fix

print("🔍 STEP-BY-STEP DIAGNOSTIC")
print("="*40)

# Step 1: Test basic database connection
print("1️⃣ Testing database connection...")
try:
    test_tables = query("SELECT name FROM sqlite_master WHERE type='table' LIMIT 5")
    print(f"✅ Connection works! Found {len(test_tables)} sample tables:")
    for table in test_tables['name']:
        print(f"   • {table}")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("Please run the database connection cell first!")

# Step 2: Check for tables with the expected columns
print("\n2️⃣ Looking for tables with department_number and gl_number...")
try:
    # Check a few common table patterns
    table_patterns = [
        'as_of_6_30_24', 'as-of-6-30-24',
        'as_of_6_30_25', 'as-of-6-30-25', 
        'general_25_26', 'general-25-26'
    ]
    
    found_tables = []
    for pattern in table_patterns:
        try:
            table_check = query(f"SELECT name FROM sqlite_master WHERE name = '{pattern}'")
            if len(table_check) > 0:
                table_name = table_check['name'][0]
                # Check if it has our target columns
                columns_check = query(f"PRAGMA table_info(`{table_name}`)")
                col_names = columns_check['name'].tolist()
                
                has_dept = any('department' in col.lower() for col in col_names)
                has_gl = any('gl_number' in col.lower() for col in col_names)
                
                if has_dept or has_gl:
                    found_tables.append({
                        'table': table_name,
                        'has_dept': has_dept,
                        'has_gl': has_gl,
                        'columns': col_names[:10]  # First 10 columns
                    })
                    print(f"   ✅ Found: {table_name}")
                    print(f"      Department column: {has_dept}")
                    print(f"      GL number column: {has_gl}")
                    print(f"      Sample columns: {col_names[:5]}...")
        except:
            pass
    
    if found_tables:
        print(f"\n✅ Found {len(found_tables)} relevant tables!")
        
        # Step 3: Create a simple padded view for the first table
        print("\n3️⃣ Creating simple test view...")
        first_table = found_tables[0]
        table_name = first_table['table']
        
        # Simple padding view creation
        simple_view_sql = f"""
        CREATE OR REPLACE VIEW `{table_name}_test_padded` AS
        SELECT *,
        CASE 
            WHEN department_number IS NOT NULL 
            THEN PRINTF('%03d', CAST(department_number AS INTEGER))
            ELSE department_number
        END as department_number_padded,
        CASE 
            WHEN gl_number IS NOT NULL 
            THEN PRINTF('%04d', CAST(gl_number AS INTEGER))
            ELSE gl_number  
        END as gl_number_padded
        FROM `{table_name}`;
        """
        
        try:
            execute(simple_view_sql)
            print(f"✅ Created test view: {table_name}_test_padded")
            
            # Test the view
            test_query = f"""
            SELECT 
                department_number, department_number_padded,
                gl_number, gl_number_padded
            FROM `{table_name}_test_padded` 
            LIMIT 5
            """
            result = query(test_query)
            print("\n📊 Before vs After comparison:")
            print(result)
            
        except Exception as e:
            print(f"❌ Error creating test view: {e}")
            print("Let's try a different approach...")
            
            # Fallback: Just show the current data
            try:
                sample_query = f"SELECT * FROM `{table_name}` LIMIT 3"
                sample_data = query(sample_query)
                print(f"\nCurrent data in {table_name}:")
                print(sample_data)
            except Exception as e2:
                print(f"❌ Can't even read the table: {e2}")
    else:
        print("❌ No tables found with the expected columns.")
        print("Let's see what tables you actually have:")
        all_tables = query("SELECT name FROM sqlite_master WHERE type='table'")
        print("Your tables:")
        for i, table in enumerate(all_tables['name'][:10], 1):
            print(f"  {i}. {table}")

except Exception as e:
    print(f"❌ Error in diagnostic: {e}")
    print("The functions might not be defined. Let's define them inline...")

# Step 4: Inline function definitions (backup)
print("\n4️⃣ Backup: Defining basic functions inline...")

def simple_padding_fix(table_name):
    """Simple function to create padded view for a specific table."""
    try:
        view_sql = f"""
        CREATE OR REPLACE VIEW `{table_name}_simple_padded` AS
        SELECT *,
        PRINTF('%03d', department_number) as dept_padded,
        PRINTF('%04d', gl_number) as gl_padded
        FROM `{table_name}`
        WHERE department_number IS NOT NULL AND gl_number IS NOT NULL;
        """
        execute(view_sql)
        print(f"✅ Created simple padded view for {table_name}")
        return True
    except Exception as e:
        print(f"❌ Failed to create view for {table_name}: {e}")
        return False

print("✅ Basic functions ready!")
print("\nTo manually fix a specific table, use:")
print("simple_padding_fix('your_table_name')")

🔍 STEP-BY-STEP DIAGNOSTIC
1️⃣ Testing database connection...
✅ Connection works! Found 5 sample tables:
   • critical_road_improvements_1
   • critical_road_improvements_2
   • capital_assets
   • rowe_cip_summary_of_projects
   • rowe_cip_funding_expenditures

2️⃣ Looking for tables with department_number and gl_number...
   ✅ Found: as_of_6_30_24
      Department column: True
      GL number column: True
      Sample columns: ['fund_name', 'fund_number', 'department_name', 'department_number', 'gl_number']...
   ✅ Found: as_of_6_30_25
      Department column: True
      GL number column: True
      Sample columns: ['fund_name', 'fund_number', 'department_name', 'department_number', 'gl_number']...
   ✅ Found: general_25_26
      Department column: True
      GL number column: True
      Sample columns: ['department_name', 'gl_number', 'description', '202324_amended_budget', '202425_amended_budget']...

✅ Found 3 relevant tables!

3️⃣ Creating simple test view...
❌ Error creating test

In [56]:
# 🔧 MINIMAL SYNTAX-SAFE DIAGNOSTIC

print("🔍 MINIMAL DIAGNOSTIC (No Complex Functions)")
print("="*50)

# Test 1: Basic connection
try:
    basic_test = query("SELECT 1 as test")
    print("✅ Database connection works!")
except Exception as e:
    print(f"❌ Connection issue: {e}")
    exit()

# Test 2: Find your tables
try:
    tables = query("SELECT name FROM sqlite_master WHERE type='table'")
    print(f"✅ Found {len(tables)} tables")
    
    # Look for specific tables
    table_names = tables['name'].tolist()
    target_tables = [name for name in table_names if 'as_of_6' in name or 'general' in name]
    
    if target_tables:
        print(f"✅ Found relevant tables: {target_tables}")
        
        # Pick the first one to work with
        work_table = target_tables[0]
        print(f"🎯 Working with: {work_table}")
        
        # Test 3: Check table structure
        try:
            cols = query(f"PRAGMA table_info(`{work_table}`)")
            col_names = cols['name'].tolist()
            print(f"✅ Table has {len(col_names)} columns")
            
            # Look for our target columns
            dept_col = None
            gl_col = None
            fund_col = None
            
            for col in col_names:
                if 'department' in col.lower():
                    dept_col = col
                if 'gl_number' in col.lower():
                    gl_col = col
                if 'fund' in col.lower():
                    fund_col = col
            
            print(f"Department column: {dept_col}")
            print(f"GL number column: {gl_col}")  
            print(f"Fund column: {fund_col}")
            
            # Test 4: Create simple view
            if dept_col and gl_col:
                view_name = f"{work_table}_SIMPLE_PADDED"
                
                simple_sql = f'''
                CREATE OR REPLACE VIEW `{view_name}` AS
                SELECT *,
                PRINTF("%03d", {dept_col}) as {dept_col}_padded,
                PRINTF("%04d", {gl_col}) as {gl_col}_padded
                FROM `{work_table}`
                '''
                
                try:
                    execute(simple_sql)
                    print(f"✅ Created view: {view_name}")
                    
                    # Test the view
                    test_result = query(f'SELECT {dept_col}, {dept_col}_padded, {gl_col}, {gl_col}_padded FROM `{view_name}` LIMIT 3')
                    print("🎉 SUCCESS! Before vs After:")
                    print(test_result)
                    
                except Exception as e:
                    print(f"❌ View creation failed: {e}")
                    print("Let me try a different approach...")
                    
                    # Fallback: Just show current data
                    sample = query(f'SELECT {dept_col}, {gl_col} FROM `{work_table}` LIMIT 3')
                    print("Current data (needs padding):")
                    print(sample)
            else:
                print("❌ Couldn't find the expected columns")
                print(f"Available columns: {col_names[:10]}")
                
        except Exception as e:
            print(f"❌ Structure check failed: {e}")
    else:
        print("❌ No relevant tables found")
        print(f"Available tables: {table_names[:10]}")
        
except Exception as e:
    print(f"❌ Table discovery failed: {e}")

print("\n🎯 MANUAL COMMANDS:")
print("If the above worked, you now have a padded view!")
print("Try queries like:")
print(f"query('SELECT * FROM your_table_SIMPLE_PADDED LIMIT 5')")

🔍 MINIMAL DIAGNOSTIC (No Complex Functions)
✅ Database connection works!
✅ Found 29 tables
✅ Found relevant tables: ['general_25_26', 'as_of_6_30_21', 'as_of_6_30_22', 'as_of_6_30_23', 'as_of_6_30_24', 'as_of_6_30_25']
🎯 Working with: general_25_26
✅ Table has 6 columns
Department column: department_name
GL number column: gl_number
Fund column: None
❌ View creation failed: near "OR": syntax error
Let me try a different approach...
Current data (needs padding):
              department_name        gl_number
0  ESTIMATED REVENUES-GENERAL  101-000-402.000
1  ESTIMATED REVENUES-GENERAL  101-000-411.000
2  ESTIMATED REVENUES-GENERAL  101-000-434.000

🎯 MANUAL COMMANDS:
If the above worked, you now have a padded view!
Try queries like:
query('SELECT * FROM your_table_SIMPLE_PADDED LIMIT 5')


## 🔄 Renaming Tables and Columns: Hyphens → Underscores

**⚠️ WARNING**: This will permanently rename your tables and columns. Consider backing up your database first!

In [18]:
# Step 1: Rename Tables (Hyphens → Underscores)

# Get list of tables that need renaming
tables_with_hyphens = """
SELECT name as old_name,
       REPLACE(name, '-', '_') as new_name
FROM sqlite_master 
WHERE type = 'table' 
    AND name LIKE '%-%'
ORDER BY name;
"""

print("Tables that will be renamed:")
tables_to_rename = query(tables_with_hyphens)
print(tables_to_rename)

# Generate ALTER TABLE statements
rename_table_commands = []
for _, row in tables_to_rename.iterrows():
    old_name = row['old_name']
    new_name = row['new_name']
    rename_cmd = f'ALTER TABLE `{old_name}` RENAME TO `{new_name}`;'
    rename_table_commands.append(rename_cmd)
    
print(f"\nGenerated {len(rename_table_commands)} table rename commands:")
for cmd in rename_table_commands:
    print(cmd)

Tables that will be renamed:
Empty DataFrame
Columns: [old_name, new_name]
Index: []

Generated 0 table rename commands:


In [19]:
# Step 2: Execute Table Renames (UNCOMMENT TO RUN)

def rename_all_tables():
    """Execute all table rename commands."""
    success_count = 0
    for cmd in rename_table_commands:
        try:
            execute(cmd)
            print(f"✓ {cmd}")
            success_count += 1
        except Exception as e:
            print(f"✗ Failed: {cmd}")
            print(f"  Error: {e}")
    
    print(f"\n✓ Successfully renamed {success_count} tables")
    return success_count

# UNCOMMENT THE LINE BELOW TO EXECUTE TABLE RENAMES:
rename_all_tables()

print("To rename tables, uncomment: rename_all_tables()")


✓ Successfully renamed 0 tables
To rename tables, uncomment: rename_all_tables()


In [20]:
# Step 3: Find Columns with Hyphens

def find_columns_with_hyphens():
    """Find all columns that contain hyphens across all tables."""
    all_tables = query("SELECT name FROM sqlite_master WHERE type='table'")
    
    columns_to_rename = []
    
    for table_name in all_tables['name']:
        try:
            # Get column info for this table
            columns_info = query(f"PRAGMA table_info(`{table_name}`)")
            
            for _, col in columns_info.iterrows():
                col_name = col['name']
                if '-' in col_name:
                    new_col_name = col_name.replace('-', '_')
                    columns_to_rename.append({
                        'table': table_name,
                        'old_column': col_name,
                        'new_column': new_col_name,
                        'type': col['type']
                    })
        except Exception as e:
            print(f"Error checking table {table_name}: {e}")
    
    return columns_to_rename

# Find all columns that need renaming
columns_with_hyphens = find_columns_with_hyphens()

print(f"Found {len(columns_with_hyphens)} columns with hyphens:")
for col in columns_with_hyphens:
    print(f"  Table: {col['table']} | Column: {col['old_column']} → {col['new_column']}")

Found 0 columns with hyphens:


In [21]:
# Step 4: Rename Columns (SQLite doesn't support ALTER COLUMN, so we recreate tables)

def rename_columns_in_table(table_name, columns_to_rename_for_table):
    """Rename columns in a table by creating new table and copying data."""
    
    # Get current table structure
    table_info = query(f"PRAGMA table_info(`{table_name}`)")
    
    # Build new CREATE TABLE statement with renamed columns
    new_columns = []
    column_mapping = {}  # old_name -> new_name
    
    for _, col in table_info.iterrows():
        old_name = col['name']
        col_type = col['type'] if col['type'] else 'TEXT'
        
        # Check if this column needs renaming
        new_name = old_name
        for rename_info in columns_to_rename_for_table:
            if rename_info['old_column'] == old_name:
                new_name = rename_info['new_column']
                break
        
        column_mapping[old_name] = new_name
        
        # Build column definition
        col_def = f"`{new_name}` {col_type}"
        if col['notnull']:
            col_def += " NOT NULL"
        if col['pk']:
            col_def += " PRIMARY KEY"
        
        new_columns.append(col_def)
    
    # Create the SQL commands
    temp_table_name = f"{table_name}_temp_rename"
    
    create_new_table = f"""
    CREATE TABLE `{temp_table_name}` (
        {', '.join(new_columns)}
    );
    """
    
    # Build INSERT statement with column mapping
    old_cols = [f"`{old_name}`" for old_name in column_mapping.keys()]
    new_cols = [f"`{new_name}`" for new_name in column_mapping.values()]
    
    copy_data = f"""
    INSERT INTO `{temp_table_name}` ({', '.join(new_cols)})
    SELECT {', '.join(old_cols)}
    FROM `{table_name}`;
    """
    
    drop_old_table = f"DROP TABLE `{table_name}`;"
    rename_temp_table = f"ALTER TABLE `{temp_table_name}` RENAME TO `{table_name}`;"
    
    return [create_new_table, copy_data, drop_old_table, rename_temp_table]

# Generate all column rename commands
def generate_all_column_rename_commands():
    """Generate all SQL commands needed to rename columns."""
    all_commands = []
    
    # Group columns by table
    tables_with_column_renames = {}
    for col_info in columns_with_hyphens:
        table = col_info['table']
        if table not in tables_with_column_renames:
            tables_with_column_renames[table] = []
        tables_with_column_renames[table].append(col_info)
    
    # Generate commands for each table
    for table_name, columns_list in tables_with_column_renames.items():
        print(f"\n--- Renaming columns in table: {table_name} ---")
        commands = rename_columns_in_table(table_name, columns_list)
        all_commands.extend(commands)
        for cmd in commands:
            print(cmd)
    
    return all_commands

# Generate the commands
if columns_with_hyphens:
    column_rename_commands = generate_all_column_rename_commands()
    print(f"\nGenerated {len(column_rename_commands)} column rename commands")
else:
    print("No columns with hyphens found!")
    column_rename_commands = []

No columns with hyphens found!


In [22]:
# Step 5: Execute All Renames (COMPLETE WORKFLOW)

def execute_complete_rename_workflow():
    """Execute the complete workflow to rename all tables and columns."""
    
    print("🔄 STARTING COMPLETE RENAME WORKFLOW")
    print("="*50)
    
    # Step 1: Rename Tables
    print("\n1️⃣ RENAMING TABLES...")
    try:
        table_success = rename_all_tables() if rename_table_commands else 0
        print(f"✅ Renamed {table_success} tables")
    except Exception as e:
        print(f"❌ Table rename failed: {e}")
        return False
    
    # Step 2: Rename Columns  
    print("\n2️⃣ RENAMING COLUMNS...")
    try:
        if column_rename_commands:
            for i, cmd in enumerate(column_rename_commands):
                execute(cmd)
                print(f"✓ Command {i+1}/{len(column_rename_commands)} completed")
            print(f"✅ Executed {len(column_rename_commands)} column rename commands")
        else:
            print("✅ No columns to rename")
    except Exception as e:
        print(f"❌ Column rename failed: {e}")
        return False
    
    print("\n🎉 WORKFLOW COMPLETED SUCCESSFULLY!")
    print("All hyphens have been replaced with underscores in table and column names.")
    print("\n✅ NO BACKTICKS NEEDED! Underscores are SQL-friendly.")
    print("Example: SELECT * FROM as_of_6_30_21")
    
    return True

# EXECUTION CONTROLS
print("🚀 READY TO RENAME ALL TABLES AND COLUMNS")
print("=" * 50)
print("This will:")
print("• Rename all table names: hyphens (-) → underscores (_)")  
print("• Rename all column names: hyphens (-) → underscores (_)")
print("• Preserve all your data")
print("• NO MORE BACKTICKS NEEDED! 🎉")
print("\n⚠️  IMPORTANT: This makes permanent changes!")
print("\nTo execute, uncomment ONE of these lines:")
print("# execute_complete_rename_workflow()  # Complete workflow")
print("# rename_all_tables()                 # Just tables")

# UNCOMMENT TO EXECUTE:
execute_complete_rename_workflow()

🚀 READY TO RENAME ALL TABLES AND COLUMNS
This will:
• Rename all table names: hyphens (-) → underscores (_)
• Rename all column names: hyphens (-) → underscores (_)
• Preserve all your data
• NO MORE BACKTICKS NEEDED! 🎉

⚠️  IMPORTANT: This makes permanent changes!

To execute, uncomment ONE of these lines:
# execute_complete_rename_workflow()  # Complete workflow
# rename_all_tables()                 # Just tables
🔄 STARTING COMPLETE RENAME WORKFLOW

1️⃣ RENAMING TABLES...
✅ Renamed 0 tables

2️⃣ RENAMING COLUMNS...
✅ No columns to rename

🎉 WORKFLOW COMPLETED SUCCESSFULLY!
All hyphens have been replaced with underscores in table and column names.

✅ NO BACKTICKS NEEDED! Underscores are SQL-friendly.
Example: SELECT * FROM as_of_6_30_21


True

In [23]:
# 🎉 AFTER RENAMING: Clean SQL Examples (No Backticks Needed!)

def show_clean_sql_examples():
    """Show examples of how clean your SQL will be after renaming."""
    
    print("✨ BEFORE vs AFTER Examples:")
    print("="*50)
    
    print("\n📋 BEFORE (with hyphens - need backticks):")
    before_examples = [
        'SELECT * FROM `as_of_6-30-25` LIMIT 5;',
        'SELECT * FROM `water_pumped_2024-2025`;',
        'SELECT `202526_recommended_budget` FROM `general_25-26`;'
    ]
    for ex in before_examples:
        print(f"  {ex}")
    
    print("\n🎉 AFTER (with underscores - clean SQL!):")
    after_examples = [
        'SELECT * FROM as_of_6_30_25 LIMIT 5;',
        'SELECT * FROM water_pumped_2024_2025;',
        'SELECT 202526_recommended_budget FROM general_25_26;'
    ]
    for ex in after_examples:
        print(f"  {ex}")
    
    print("\n✅ Benefits:")
    print("  • No more backticks needed!")
    print("  • Cleaner, more readable SQL")
    print("  • Standard SQL naming conventions")
    print("  • Works in all SQL tools without escaping")

# Run this after renaming to see the improvements
show_clean_sql_examples()

✨ BEFORE vs AFTER Examples:

📋 BEFORE (with hyphens - need backticks):
  SELECT * FROM `as_of_6-30-25` LIMIT 5;
  SELECT * FROM `water_pumped_2024-2025`;
  SELECT `202526_recommended_budget` FROM `general_25-26`;

🎉 AFTER (with underscores - clean SQL!):
  SELECT * FROM as_of_6_30_25 LIMIT 5;
  SELECT * FROM water_pumped_2024_2025;
  SELECT 202526_recommended_budget FROM general_25_26;

✅ Benefits:
  • No more backticks needed!
  • Cleaner, more readable SQL
  • Standard SQL naming conventions
  • Works in all SQL tools without escaping


In [ ]:
# ⚡ QUICK FIX: Fast Leading Zero Padding (Won't Get Stuck!)

print("⚡ QUICK PADDING FIX - LIGHTWEIGHT VERSION")
print("="*50)

# Only check specific known tables to avoid scanning everything
known_tables = [
    'as_of_6_30_24', 'as-of-6-30-24',
    'as_of_6_30_25', 'as-of-6-30-25', 
    'general_25_26', 'general-25-26',
    'as_of_6_30_23', 'as-of-6-30-23'
]

print("🎯 Checking specific tables only (won't scan entire database)...")

success_count = 0

for table_name in known_tables:
    try:
        # Quick check if table exists
        table_check = query(f"SELECT name FROM sqlite_master WHERE name = '{table_name}' LIMIT 1")
        
        if len(table_check) > 0:
            actual_table = table_check['name'][0]
            print(f"\n📋 Found: {actual_table}")
            
            # Quick column check
            cols = query(f"PRAGMA table_info(`{actual_table}`)")
            col_names = cols['name'].tolist()
            
            dept_col = next((col for col in col_names if 'department' in col.lower()), None)
            gl_col = next((col for col in col_names if 'gl_number' in col.lower()), None)
            
            if dept_col or gl_col:
                # Create simple padded view
                view_name = f"{actual_table}_PADDED"
                
                view_sql = f"""
CREATE OR REPLACE VIEW `{view_name}` AS
SELECT *,
{f"PRINTF('%03d', CAST(`{dept_col}` AS INTEGER)) as {dept_col}_padded," if dept_col else ""}
{f"PRINTF('%04d', CAST(`{gl_col}` AS INTEGER)) as {gl_col}_padded" if gl_col else ""}
FROM `{actual_table}`;
"""
                
                # Clean up SQL (remove empty lines)
                view_sql = view_sql.replace(',\n\nFROM', '\nFROM').replace('SELECT *,\n\nFROM', 'SELECT *\nFROM')
                
                execute(view_sql)
                print(f"   ✅ Created: {view_name}")
                
                # Quick test (just 1 row to avoid slowdown)
                if dept_col and gl_col:
                    sample = query(f"SELECT {dept_col}, {gl_col} FROM `{view_name}` LIMIT 1")
                    print(f"   📊 Sample: {sample.iloc[0].to_dict()}")
                
                success_count += 1
            else:
                print(f"   ⚠️  No dept/GL columns found")
                
    except Exception as e:
        print(f"   ❌ Error with {table_name}: {str(e)[:50]}...")
        continue

print(f"\n🎉 COMPLETED! Created {success_count} padded views.")
print("\n🎯 NOW YOU CAN USE:")
if success_count > 0:
    print("   • query('SELECT * FROM as_of_6_30_24_PADDED LIMIT 5')")
    print("   • All department numbers: 001, 002, 003...")
    print("   • All GL numbers: 0001, 0012, 0123...")
else:
    print("   • Check table names with: query('SELECT name FROM sqlite_master WHERE type=\"table\"')")

print("\n⚡ This version is fast and won't hang!")